In [226]:
# Riddler Classic 2022-08-19
# https://fivethirtyeight.com/features/can-you-find-your-pills/
# It follows the Markov method I used in https://github.com/khgiddon/misc/blob/main/riddler_2020_10_02_notebook.ipynb

from scipy.stats import hypergeom
import pandas as pd

transitions = set()
queue = set(((15,0),))

def compute_transitions(full_pills,half_pills):
    
    # Transitions tuple represented by: (current_full_pills,current_half_pills,new_full_pills,new_half_pills,p)
    
    # Probabilities of picking up 0, 1, 2 half pills
    [a,b,c] = [full_pills+half_pills, half_pills, 2]
    p1, p2, p3 = (hypergeom(a,b,c).pmf(n) for n in range(3))
    
    # If picked up 2 half pills, what happens next
    if half_pills >= 2:
        [a,b,c] = [full_pills+half_pills-2, half_pills-2, 1]
        p4 = hypergeom(a,b,c).pmf(1) # Picked up one half pill
    else:
        p4 = 0
    
    # Check if new states are valid, and then if so append to transitions list
    if half_pills - 3 >= 0 and half_pills-3 + full_pills > 0:
        transitions.add((full_pills,half_pills,full_pills,half_pills-3,p3*p4))
        queue.add((full_pills,half_pills-3)) # New state to test

    if full_pills-2 >= 0 and half_pills+1 + full_pills-2 > 0:              
        transitions.add((full_pills,half_pills,full_pills-2,half_pills+1,p1)) # Picked up 2 full pills
        queue.add((full_pills-2,half_pills+1))
                     
    if full_pills-1 >= 0 and half_pills-1 >= 0  and half_pills-1 + full_pills-1 > 0:
        transitions.add((full_pills,half_pills,full_pills-1,half_pills-1,p2 + (p3*(1-p4)))) # Picked up 1 half pill and 1 full pill ; this also happens if you pick up 2 half pills then a full pill
        queue.add((full_pills-1,half_pills-1))
    
while len(queue) > 0:
    full_pills, half_pills = queue.pop()
    if full_pills + half_pills > 0:
        compute_transitions(full_pills, half_pills)

# Clean and convert to transition matrix
df = pd.DataFrame(transitions,columns = ['full_pills','half_pills', 'new_full_pills','new_half_pills','p'])
df['id'] = df['full_pills'].astype(str).str.zfill(2) + '_' + df['half_pills'].astype(str).str.zfill(2)
df['id_new'] = df['new_full_pills'].astype(str).str.zfill(2) + '_' + df['new_half_pills'].astype(str).str.zfill(2)

# Label transition to absorbing states differently so they sort last
df['id'] = df['id'].str.replace('00_03','-00_03')
df['id_new'] = df['id_new'].str.replace('00_03','-00_03')
df['id'] = df['id'].str.replace('01_01','-01_01')
df['id_new'] = df['id_new'].str.replace('01_01','-01_01')

# Create absorbing states
df1 = pd.DataFrame(columns=df.columns)
df1.loc[0] = [0,0,0,0,1,'-00_03','-00_03']
df2 = pd.DataFrame(columns=df.columns)
df2.loc[0] = [0,0,0,0,1,'-01_01','-01_01']
df = pd.concat([df,df1,df2],ignore_index=True)

# Rectangularize 
df_pivot = df.pivot_table(index='id', columns='id_new', values='p',fill_value=0,dropna=False)      
df_pivot['15_00'] = 0

# Order absorbing states last
df_pivot = df_pivot.sort_index(ascending=False)
df_pivot = df_pivot.sort_index(ascending=False,axis=1)

# Markov chain math
matrix = df_pivot.to_numpy()
I = np.eye(len(matrix) - 2)  
Q = matrix[:-2, :-2]
R = matrix[:-2,-2:]

# And the answer is ...
print(round(np.matmul(np.linalg.inv(I - Q), R)[0][0],10))

0.7901197017
